# Toxicity metrics data generation
In this notebook I generate toxicity metrics with the Detoxify library which is used to measure toxicity of texts, in our case tweets. 
This is meant as a suplementary approach to the Perpective API since we are limited by the number of queries when using it.

Make sure to install CUDA achieves at least 5x speed up.

In [ ]:
%reload_ext autoreload
%autoreload 2

import os 
import sys
import pandas as pd
import numpy as np
import plotly 
import plotly.graph_objects as go
import time

import nltk
from detoxify import Detoxify
# nltk.download('stopwords')

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../../")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"

In [ ]:
import torch
# clear memory to reduce memory errors
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

# test if cuda is available, it has to be otherwise slow asf, 33 hours for 1.2 million tweets 
device = torch.device("cuda")
cuda_present = torch.cuda.is_available()
print(f"Cuda present: {cuda_present}")

# load the model
model = Detoxify('original', device="cuda")

In [ ]:
tweets_df = pd.read_csv("./data/twitter_1_million_tweet_dump_29_12_2022.csv")
total_len = len(tweets_df.index)
tweets_df = tweets_df[tweets_df["lang"] == "en"]
print(f"Removed {total_len - len(tweets_df.index)} tweets out of {len(tweets_df.index)}, since they were not in English")
tweets_df

## Generating toxicity scores for each tweet
The code below needed 19092 seconds (5.3 hours) to run the last time, with CUDA.

In [ ]:
csv_file_n = "toxicity_temp_v3.csv"
# generating toxicity scores for each tweet
start_time = time.time() 
toxicity_df = pd.DataFrame(columns=["toxicity", "severe_toxicity", "obscene", "threat", "insult", "identity_attack"])
toxicity_df.to_csv(csv_file_n)

content_list = tweets_df["text"].to_list()
step = 50
for i in range(0, len(tweets_df.index), step):
    print(f"At row: {i}")
    # everything in one row to reduce memory consumption
    if i % 500 == 0:
        torch.cuda.empty_cache()
        toxicity_df.to_csv(csv_file_n, mode='a', header=False)
        print("Cleared GPU cache and saved to file")
        toxicity_df = pd.DataFrame(columns=["toxicity", "severe_toxicity", "obscene", "threat", "insult", "identity_attack"])
        
    toxicity_df = pd.concat([toxicity_df, pd.DataFrame(model.predict(content_list[i:i+step]))], ignore_index=True)

toxicity_df.to_csv(csv_file_n, mode='a', header=False)
print(f"Execution took: {time.time() - start_time:.2f} seconds")
display(toxicity_df) 

In [ ]:
# load the csv with the toxicity data
tox_df = pd.read_csv("toxicity_temp_v2.csv")
tox_df = tox_df.drop("Unnamed: 0", axis=1)
display(tox_df)

In [ ]:
# merge tweets with toxicity and save it to a file
merged_toxic_df = pd.merge(tweets_df, tox_df, left_index=True, right_index=True)
merged_toxic_df.to_csv("merged_tweets_and_toxicity_31_12.csv")
display(merged_toxic_df)